In [1]:
import pandas as pd
import re

In [2]:
# Загрузка корпуса с разметкой
corpus_path = 'parsed_blocks_with_nlp.csv'
corpus = pd.read_csv(corpus_path, sep=';')

In [3]:
corpus.head()

,Meta,Sentence,Tokens,Lemmas,POS
0,Письмо первое. БОЛЬШОЕ В МАЛОМ,В материальном мире большое не уместишь в малом.,В материальном мире большое не уместишь в малом .,в материальный мир большой не уместить в малый .,ADP ADJ NOUN ADJ PART VERB ADP ADJ PUNCT
1,Письмо первое. БОЛЬШОЕ В МАЛОМ,В сфере же духовных ценностей не так: в малом...,В сфере же духовных ценностей не так : в мало...,в сфера же духовный ценностей не так : в малы...,ADP NOUN PART ADJ NOUN PART ADV PUNCT ADP ADJ ...
2,Письмо первое. БОЛЬШОЕ В МАЛОМ,"Если есть у человека великая цель, то она долж...","Если есть у человека великая цель , то она дол...","если быть у человек великий цель , то она долж...",SCONJ VERB ADP NOUN ADJ NOUN PUNCT SCONJ PRON ...
3,Письмо первое. БОЛЬШОЕ В МАЛОМ,Надо быть честным в незаметном и случайном: т...,Надо быть честным в незаметном и случайном : ...,надо быть честный в незаметный и случайный : ...,ADV AUX ADJ ADP ADJ CCONJ ADJ PUNCT ADV PART A...
4,Письмо первое. БОЛЬШОЕ В МАЛОМ,"Большая цель охватывает всего человека, сказыв...","Большая цель охватывает всего человека , сказы...","больший цель охватывать весь человек , сказыва...",ADJ NOUN VERB DET NOUN PUNCT VERB ADP DET DET ...


In [41]:
# Функция для поиска по корпусу с учётом различных типов запросов
def search(query):
    results = []

    # Разбиваем запрос на части (токены)
    tokens = query.split()
    
    for index, row in corpus.iterrows():
        sentence = row['Sentence']
        meta = row['Meta']
        tokens_list = row['Tokens'].split()
        lemmas_list = row['Lemmas'].split()
        pos_list = row['POS'].split()
        
        # Флаг, указывающий на соответствие предложению
        match = True
        
        # Указываем позицию для проверки
        current_positions = []

        # Обработка каждого токена запроса
        for token in tokens:
            token_found = False
            
            if "+" in token:  # запрос лемма+POS, например "знать+NOUN"
                word, pos = token.split("+")
                for i in range(len(lemmas_list)):
                    if lemmas_list[i] == word and pos_list[i] == pos:
                        current_positions.append(i)
                        token_found = True
                        break

            elif token.startswith('"') and token.endswith('"'):  # запрос точной формы в кавычках
                word_form = token.strip('"')
                for i in range(len(tokens_list)):
                    if tokens_list[i] == word_form:
                        current_positions.append(i)
                        token_found = True
                        break
                
            elif re.match(r'^[A-Z]+$', token):  # POS-тег, например "NOUN"
                for i in range(len(pos_list)):
                    if pos_list[i] == token:
                        current_positions.append(i)
                        token_found = True
                        break
                
            else:  # Обычная лемма или словоформа, например "поиск"
                for i in range(len(lemmas_list)):
                    if lemmas_list[i] == token or tokens_list[i] == token:
                        current_positions.append(i)
                        token_found = True
                        break
            
            if not token_found:
                match = False
                break
        
        # Проверяем, что расстояние между токенами равно 1
        if match:
            for i in range(len(current_positions) - 1):
                if abs(current_positions[i] - current_positions[i + 1]) > 1:
                    match = False
                    break
        
        # Если все токены в запросе соответствуют предложению, добавляем предложение в результаты
        if match:
            results.append(f"{sentence} - {meta}")

    # Вывод результатов
    if results:
        print("Найденные предложения:\n")
        for result in results:
            print(result)
    else:
        print("Ничего не найдено.")

In [43]:
search('ADJ любовь')

Найденные предложения:

Настоящая любовь к людям, к памятникам ли никогда не остается без ответа. - Письмо сорок третье. ЕЩЕ О ПАМЯТНИКАХ ПРОШЛОГО
